In [1]:
## Spell checker from Peter Norvig. Not perfect, but works.
import re, collections

def words(text): return re.findall('[a-z]+', text.lower()) 

def train(features):
    model = collections.defaultdict(lambda: 1)
    for f in features:
        model[f] += 1
    return model

NWORDS = train(words(open('C:/Users/kenndanielso/Documents/Github/mcnulty_yelp/data/big.txt').read()))

alphabet = 'abcdefghijklmnopqrstuvwxyz'

def edits1(word):
   splits     = [(word[:i], word[i:]) for i in range(len(word) + 1)]
   deletes    = [a + b[1:] for a, b in splits if b]
   transposes = [a + b[1] + b[0] + b[2:] for a, b in splits if len(b)>1]
   replaces   = [a + c + b[1:] for a, b in splits for c in alphabet if b]
   inserts    = [a + c + b     for a, b in splits for c in alphabet]
   return set(deletes + transposes + replaces + inserts)

def known_edits2(word):
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1) if e2 in NWORDS)

def known(words): return set(w for w in words if w in NWORDS)

def correct(word):
    candidates = known([word]) or known(edits1(word)) or known_edits2(word) or [word]
    return max(candidates, key=NWORDS.get)

In [2]:
## Import all modules needed
## lem is lemmatizing the words
def lem(x):
    sentence = ''
    for x in re.sub('[^a-z0-9 ]','',x).split():   
        sentence = sentence + ' ' + lemmatizer.lemmatize(correct(x))
    count = 0
    return sentence

import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.stem import WordNetLemmatizer
%matplotlib inline
from nltk import word_tokenize
class LemmaTokenizer(object):
     def __init__(self):
         self.wnl = WordNetLemmatizer()
     def __call__(self, doc):
         return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

from nltk.corpus import stopwords

stops = set(stopwords.words('english'))

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = CountVectorizer(min_df=100,stop_words=stops,max_features=1000,ngram_range=(1,1),tokenizer=LemmaTokenizer())
tfvectorizer = TfidfVectorizer(stop_words=stops,min_df=100,max_features=1000,ngram_range=(1,1),tokenizer=LemmaTokenizer())

import pandas as pd
import pickle
with open("C:/Users/kenndanielso/Documents/Github/mcnulty_yelp/data/review_business_df.pkl", 'rb') as picklefile: 
    review_business_df = pickle.load(picklefile)

In [3]:
sample_df = review_business_df.dropna().reset_index(drop=True)

In [4]:
short_df = sample_df.iloc[:,0:2]

In [5]:
short_df['spell'] = short_df['text'].apply(lambda x: x.lower())

In [6]:
# short_df['s_spell_lem'] = short_df['spell'].apply(lem).apply(correct)

In [7]:
new_df = short_df[['spell','stars_x']]

In [8]:
corp = list(new_df.spell)

In [9]:
feature = vectorizer.fit_transform(corp)

In [10]:
featuredf = pd.DataFrame(feature.A, columns=vectorizer.get_feature_names())

In [11]:
new_df = pd.concat((new_df,featuredf),axis=1)

In [12]:
new_df['stars_x'] = new_df['stars_x'].astype('category')
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1363242 entries, 0 to 1363241
Columns: 1002 entries, spell to yummy
dtypes: category(1), int64(1000), object(1)
memory usage: 10.2+ GB


In [18]:
new_df['stars_x'] = new_df['stars_x'].astype('category')

In [20]:
new_df.head()

,spell,stars_x,!,#,$,%,&,','','d,...,write,wrong,yeah,year,yelp,yes,yet,young,yum,yummy
0,"mr hoagie is an institution. walking in, it do...",4,0,0,0,0,0,0,0,0,...,0,0,0,3,0,0,0,0,0,0
1,excellent food. superb customer service. i mis...,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,yes this place is a little out dated and not o...,5,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
3,this place was delicious!! my parents saw a r...,5,6,0,2,0,3,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,can't miss stop for the best fish sandwich in ...,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
import pickle
with open('C:/Users/kenndanielso/Documents/Github/mcnulty_yelp/data/kenn_df.pkl', 'wb') as picklefile: 
    pickle.dump(new_df, picklefile)

OverflowError: cannot serialize a bytes object larger than 4 GiB

# Model testing

In [14]:
##Split into train and test at 75/25
from sklearn.cross_validation import train_test_split 
train, test = train_test_split(new_df,test_size = 0.25)

In [15]:
##Split X & Y
X_train = train.iloc[:,2:]
Y_train = train.iloc[:,1]
X_test = test.iloc[:,2:]
Y_test = test.iloc[:,1]

In [23]:
from sklearn.metrics import confusion_matrix

## NaiveBayes
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB().fit(X_train,Y_train)
nb_Y_pred = nb.predict(X_test)
print("NB Accuracy: ",np.mean(nb_Y_pred == np.array(Y_test)))
print(confusion_matrix(Y_test,nb_Y_pred,labels=['1','2','3','4','5']))

NB Accuracy:  0.564242351333
[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]


In [ ]:
## Logistic
from sklearn.linear_model import LogisticRegression
log = LogisticRegression().fit(X_train,Y_train)
log_Y_pred = log.predict(X_test)
print("Logistc Accuracy: ",np.mean(log_Y_pred == np.array(Y_test)))
print(confusion_matrix(Y_test,log_Y_pred,labels=['1','2','3','4','5']))

In [ ]:
## Linear SVC
from sklearn.svm import SVC
svcl = SVC(kernel='linear').fit(X_train,Y_train)
svcl_Y_pred = svcl.predict(X_test)
print("SVC Linear Accuracy: ",np.mean(svcl_Y_pred == np.array(Y_test)))
print(confusion_matrix(Y_test,svcl_Y_pred,labels=['1','2','3','4','5']))

In [ ]:
## RBF SVC
from sklearn.svm import SVC
svcrbf = SVC(kernel='rbf', gamma=1).fit(X_train,Y_train)
svcrbf_Y_pred = svcrbf.predict(X_test)
print("SVC RBF Accuracy: ",np.mean(svcrbf_Y_pred == np.array(Y_test)))
print(confusion_matrix(Y_test,svcrbf_Y_pred,labels=['1','2','3','4','5']))

In [ ]:
## Decision Tree
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier().fit(X_train,Y_train)
dt_Y_pred = dt.predict(X_test)
print("Decision Tree Accuracy: ",np.mean(dt_Y_pred == np.array(Y_test)))
print(confusion_matrix(Y_test,dt_Y_pred,labels=['1','2','3','4','5']))

In [ ]:
## Random Forests
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier().fit(X_train,Y_train)
rf_Y_pred = rf.predict(X_test)
print("Random Forests Accuracy: ",np.mean(rf_Y_pred == np.array(Y_test)))
print(confusion_matrix(Y_test,rf_Y_pred,labels=['1','2','3','4','5']))